In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents (1).json


In [ ]:
with open("intents.json") as content:
  data = json.load(content)

tags = []
patterns = []
responses = {}
for intent in data["intents"]:
  responses[intent["tag"]] = intent["responses"]
  for lines in intent["patterns"]:
    patterns.append(lines)
    tags.append(intent["tag"])

data = pd.DataFrame({"patterns":patterns,
                     "tags":tags})
print(data)

In [ ]:
import string
data["patterns"] = data["patterns"].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data["patterns"] = data["patterns"].apply(lambda wrd: "".join(wrd))

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data["patterns"])
train = tokenizer.texts_to_sequences(data["patterns"])

from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data["tags"])

In [ ]:
pattern_shape = x_train.shape[1]
print("maximum number of elements in the pattern data: ", pattern_shape)

vocabulary = len(tokenizer.word_index)
print("number of unique words: ",vocabulary)

output_length = le.classes_.shape[0]
print("output length: ",output_length)

In [ ]:
i = Input(shape = (pattern_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences = True)(x)
x = Flatten()(x)
x = Dense(output_length,activation = "softmax")(x)
model  = Model(i, x)

model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

train = model.fit(x_train, y_train, epochs=200)

In [ ]:
import random

while True:
  texts_p = []
  prediction_input = input("You : ")

  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = "".join(prediction_input)
  texts_p.append(prediction_input)

  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input], pattern_shape)

  output = model.predict(prediction_input)
  output = output.argmax()

  response_tag = le.inverse_transform([output])[0]
  print("Bot: ", random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break